In [ ]:
# -------------------------------
# Step 0: Install & Import
# -------------------------------
!pip install opencv-python matplotlib scikit-learn tensorflow tqdm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import cv2

# -------------------------------
# Step 1: Load CIFAR-10 dataset
# -------------------------------
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()

# Normalize
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# Convert to grayscale
def rgb2gray(images):
    return np.expand_dims(np.dot(images[...,:3], [0.2989, 0.5870, 0.1140]), axis=-1)

gray_train = rgb2gray(x_train)
gray_test  = rgb2gray(x_test)

# -------------------------------
# Step 2: Simulate User Hints
# -------------------------------
def generate_hint_mask(gray, color, hint_count=10):
    hint_mask = np.zeros_like(color)
    for _ in range(hint_count):
        y, x = np.random.randint(0, gray.shape[0]), np.random.randint(0, gray.shape[1])
        hint_mask[y, x] = color[y, x]
    return hint_mask

hints_train = np.array([generate_hint_mask(g, c) for g, c in zip(gray_train, x_train)])
hints_test  = np.array([generate_hint_mask(g, c) for g, c in zip(gray_test, x_test)])

# -------------------------------
# Step 3: Build Model
# -------------------------------
def build_unet(input_shape=(32, 32, 1)):
    input_img = layers.Input(shape=input_shape)
    input_hint = layers.Input(shape=(32, 32, 3))
    x = layers.Concatenate()([input_img, input_hint])

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    output = layers.Conv2D(3, (1, 1), activation='sigmoid', padding='same')(x)
    return models.Model(inputs=[input_img, input_hint], outputs=output)

model = build_unet()
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

# -------------------------------
# Step 4: Train Model
# -------------------------------
history = model.fit(
    [gray_train, hints_train], x_train,
    validation_data=([gray_test, hints_test], x_test),
    epochs=20,
    batch_size=32
)

# -------------------------------
# Step 5: Evaluate
# -------------------------------
predictions = model.predict([gray_test, hints_test])
# Quantize predictions to 16 color bins and compare with true
true_flat = np.round(x_test.reshape(-1, 3) * 15)
pred_flat = np.round(predictions.reshape(-1, 3) * 15)

print("\nClassification Report (approximate colors):")
print(classification_report(true_flat.astype(int).flatten(), pred_flat.astype(int).flatten()))

# -------------------------------
# Step 6: Save model
# -------------------------------
model.save("saved_model/user_guided_colorization_model.keras")
model.save_weights("user_guided_colorization_weights.h5")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32, 4) │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │      2,368 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 16,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 16, 16,    │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 32, 32,    │          0 │ conv2d_1[0][0]    │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │     73,792 │ up_sampling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 32, 32, 3) │        195 │ conv2d_2[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 150,211 (586.76 KB)

 Trainable params: 150,211 (586.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 751s 479ms/step - accuracy: 0.4767 - loss: 0.0105 - val_accuracy: 0.5071 - val_loss: 0.0047
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 795s 475ms/step - accuracy: 0.5801 - loss: 0.0045 - val_accuracy: 0.6207 - val_loss: 0.0042
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 802s 475ms/step - accuracy: 0.6040 - loss: 0.0042 - val_accuracy: 0.6085 - val_loss: 0.0043
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 741s 474ms/step - accuracy: 0.6133 - loss: 0.0040 - val_accuracy: 0.6190 - val_loss: 0.0039
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 743s 475ms/step - accuracy: 0.6165 - loss: 0.0039 - val_accuracy: 0.6383 - val_loss: 0.0039
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 742s 474ms/step - accuracy: 0.6210 - loss: 0.0038 - val_accuracy: 0.6417 - val_loss: 0.0038
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 741s 474ms/step - accuracy: 0.6230 - loss: 0.0038 - val_accuracy: 0.6285 - val_loss: 0.0041
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 739s 472ms/step - ac

FileNotFoundError: [Errno 2] No such file or directory: 'saved_model/user_guided_colorization_model.keras'